In [3]:
source('utils.r')
source('selinf_functions.r')
source('metrics.r')
source('data_generator.r')
source('cov_matrix.r')
library('lmmlasso')

In [12]:
library(lme4)
library(lmerTest)
# use the ham data and use scaled information liking
# as response
ham$Informed.liking <- scale(ham$Informed.liking)

# We first define a function to fit a model based on response
# This function is usually not required but can be 
# specified in order to define the selection_function
# as a function of the model instead of as a function
# of the response vector
modFun <- function(y)
{
  ham$y <- y
  lmer(y ~ Gender + Information * Product + (1 | Consumer) +
  (1 | Product),REML = FALSE, data=ham)
  
  }

# define the selection_function:
# here this is done as function of a model
# which, in combination with modFun, can then
# be used as function 
selFun <- function(mod) step(mod, reduce.fixed = FALSE)

# define a function which extracts the results 
# of the selection procedure
extractSelFun <- function(this_mod){

this_mod <- attr(this_mod, "model")
if(class(this_mod)=="lm") 
  return(attr(this_mod$coefficients, "names")) else
    return(c(names(fixef(this_mod)), 
             names(getME(this_mod, "theta"))))

}

# Now we run the initial model selection on the 
# orginal data, which is a 
# backward elimination of non-significant effects:
(step_result <- selFun(modFun(ham$Informed.liking)))
attr(step_result, "model")
# Elimination tables for random- and fixed-effect terms:
(sel <- extractSelFun(step_result))

# Now we can define the function checking the congruency
# with the original selection
checkFun <- function(yb){ 

 this_mod <- modFun(yb)
 setequal( extractSelFun(selFun(this_mod)), sel )
 
 }

# and compute valid p-values conditional on the selection
# (this takes some time and will produce a lot of warnings)
res <- mocasin(attr(step_result, "model"), this_y = ham$Informed.liking,
          checkFun = checkFun, which = 1:4, nrSamples = 50, trace = FALSE, varForSampling = 'minMod')

print(res)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"


Warning message:
"Model failed to converge with 1 negative eigenvalue: -2.4e-07"


Backward reduced random-effect table:

               Eliminated npar  logLik    AIC    LRT Df Pr(>Chisq)    
<none>                      12 -901.24 1826.5                         
(1 | Product)           1   11 -901.24 1824.5  0.000  1          1    
(1 | Consumer)          0   10 -919.61 1859.2 36.737  1  1.352e-09 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Backward reduced fixed-effect table:
Degrees of freedom method: Satterthwaite 

                    Eliminated  Sum Sq Mean Sq NumDF DenDF F value Pr(>F)
Gender                       0 0.72398 0.72398     1    79  0.8789 0.3514
Information:Product          0 1.95434 0.65145     3   560  0.7909 0.4993

Model found:
y ~ Gender + Information + Product + (1 | Consumer) + Information:Product

Linear mixed model fit by REML ['lmerModLmerTest']
Formula: 
y ~ Gender + Information + Product + (1 | Consumer) + Information:Product
   Data: ham
REML criterion at convergence: 1802.475
Random effects:
 Groups   Name        Std.Dev.
 Consumer (Intercept) 0.3946  
 Residual             0.9076  
Number of obs: 648, groups:  Consumer, 81
Fixed Effects:
          (Intercept)                Gender2           Information2  
              0.05032               -0.10596                0.06962  
             Product2               Product3               Product4  
             -0.35879                0.06426                0.12852  
Information2:Product2  Information2:Product3  Information2:Product4  
              0.10710                0.11781               -0.15530  

[1] "(Intercept)"           "Gender2"               "Information2"         
 [4] "Product2"              "Product3"              "Product4"             
 [7] "Information2:Product2" "Information2:Product3" "Information2:Product4"
[10] "Consumer.(Intercept)"

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
" Hessian is numerically singular: parameters are not uniquely determined"
Warning message:
"Model failed to converge with 1 negative eigenvalue: -2.4e-07"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model is nearly unidentifiable: large eigenvalue ratio
 - Rescale variables?"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$pa

  nrsurv       tstat        pval        cil         ciu
1     50  0.05031780 0.893029368 -0.7836708  0.77861170
2     50 -0.10596002 0.975747190 -1.2040008  1.40766456
3     50  0.06961554 0.745476760 -0.2812720  0.36529227
4     50 -0.35878777 0.009630103 -0.6463054 -0.09358789
